In [37]:
# Imports
import pickle, json
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Importing, opening pickled array, size brought down from 20.4 GBs to .5 GBs pre-pickling
with open('PreProcessing/kickXY.pkl', 'rb') as fp:
	base = pickle.load(fp)

In [38]:
# Key for array column names
# key = [0'unid', 1'timeline', 2'goalUSD', 3'category', 4'text', 5'state']

# Turning array input from scraped data into dataframe
bigDF = pd.DataFrame(data=base)

# Splitting category 'slug' into category & subcategory
bigDF[3] = bigDF[3].str.split('/', n=1)
bigDF[6] = [bigDF[3][i][-1] for i in range(len(bigDF))]
bigDF[3] = [bigDF[3][i][0] for i in range(len(bigDF))]

# Dropping 'live', 'canceled', 'suspended' records and removing dupes (3.6M->221K)
states = ['live', 'canceled', 'suspended']
for i in range(len(states)):
    bigDF.drop(bigDF[bigDF[5] == states[i]].index, inplace=True)

bigDFunique = bigDF.drop_duplicates(keep='last')

# Dropping unique_id column post dupe-removal
bigDFunique.drop(columns=0, inplace=True)

/Users/rowenwitt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [18]:
# Saving cleaned, de-duped, state-simplified csv (3.6M -> 221K)
bigDFunique.to_csv('cleanKick.csv')

In [39]:
bigDFunique

,1,2,3,4,5,6
21294,30,7895.704536,publishing,粵語文學期刊\nCantonese Literature Periodical,successful,literary journals
21976,24,4008.273100,comics,"A survival horror about love, acceptance, deat...",successful,graphic novels
29021,60,11000.000000,comics,A graphic novel and unique take on Anthology H...,failed,graphic novels
44904,31,1558.756540,comics,An anthology with 3 self-contained shorts of T...,successful,anthologies
45339,31,5623.373450,games,The world's first truly travel optimized playi...,successful,playing cards
...,...,...,...,...,...,...
3658378,17,600.000000,art,Back this project! Or don't. It doesn't matte...,failed,conceptual art
3658379,30,10000.000000,publishing,How do you make friends as an adult? I created...,failed,young adult
3658380,20,3100.000000,art,A 32 card oracle deck filled with ghosts & spi...,successful,illustration
3658381,30,5000.000000,publishing,Mars Awaits! Each piece of artwork shows one s...,failed,calendars


In [44]:
# Label encode category, state, subCategory
LeCat = LabelEncoder()
LeSubCat = LabelEncoder()
LeState = LabelEncoder()

# Fit encoders on respective columns
LeCat.fit(bigDFunique[3])
LeSubCat.fit(bigDFunique[6])
LeState.fit(bigDFunique[5])

# Apply encodings to columns
bigDFunique[3] = LeCat.transform(bigDFunique[3])
bigDFunique[6] = LeSubCat.transform(bigDFunique[6])
bigDFunique[5] = LeState.transform(bigDFunique[5])

<ipython-input-44-e6fc0a584793>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[3] = LeCat.transform(bigDFunique[3])
<ipython-input-44-e6fc0a584793>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[6] = LeSubCat.transform(bigDFunique[6])
<ipython-input-44-e6fc0a584793>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

In [45]:
# create dict maps
def ones(imp):
    out = []
    for i in range(len(imp)):
        out.append(i)
    
    dmap = dict(zip(out, imp))
    return dmap

StateMap = ones(LeState.classes_)
CatMap = ones(LeCat.classes_)
SubCatMap = ones(LeSubCat.classes_)

In [59]:
SubCatMap

{0: '3d printing',
 1: 'academic',
 2: 'accessories',
 3: 'action',
 4: 'animals',
 5: 'animation',
 6: 'anthologies',
 7: 'apparel',
 8: 'apps',
 9: 'architecture',
 10: 'art',
 11: 'art books',
 12: 'audio',
 13: 'bacon',
 14: 'blues',
 15: 'calendars',
 16: 'camera equipment',
 17: 'candles',
 18: 'ceramics',
 19: "children's books",
 20: 'childrenswear',
 21: 'chiptune',
 22: 'civic design',
 23: 'classical music',
 24: 'comedy',
 25: 'comic books',
 26: 'comics',
 27: 'community gardens',
 28: 'conceptual art',
 29: 'cookbooks',
 30: 'country & folk',
 31: 'couture',
 32: 'crafts',
 33: 'crochet',
 34: 'dance',
 35: 'design',
 36: 'digital art',
 37: 'diy',
 38: 'diy electronics',
 39: 'documentary',
 40: 'drama',
 41: 'drinks',
 42: 'electronic music',
 43: 'embroidery',
 44: 'events',
 45: 'experimental',
 46: 'fabrication tools',
 47: 'faith',
 48: 'family',
 49: 'fantasy',
 50: "farmer's markets",
 51: 'farms',
 52: 'fashion',
 53: 'festivals',
 54: 'fiction',
 55: 'film & vid

In [46]:
# Instantiating Tokenizer
tok = Tokenizer()

# Fitting tokenizer on df text array
tok.fit_on_texts(texts=bigDFunique[4])

# Getting word index from tokenizer
dex = tok.word_index

# Keep work index to 1000 words
tok.num_words = 1000

# Creating sequences from df text array
seqs = tok.texts_to_sequences(bigDFunique[4])
max_features = len(dex.values()) + 1
maxlen = 43

# Saving tokenizer to pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Padding sequence length
bigX = sequence.pad_sequences(seqs, maxlen)

# Creating list of reconstructed text
recons = tok.sequences_to_texts(seqs)

In [50]:
bigX[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)

In [60]:
bigDFunique.shape

(211038, 6)

In [27]:
# Giving text encodings (bigX) own column 7-49
for j in range(7, (50)):
    f = j-7
    bigDFunique[j] = [bigX[i][f] for i in range(len(bigDFunique))]

<ipython-input-27-b0ee71e790d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bigDFunique[j] = [bigX[i][f] for i in range(len(bigDFunique))]


In [28]:
# dropping out column 4 (unencoded text), re-naming columns, renaming df to df
df = bigDFunique.drop(columns=[4])

newColumns = [i for i in range(48)]
df.columns = newColumns

# Splitting into X and Y (4 = success/failure)
target = 3
X = df.drop(columns=target)
Y = df[target]

In [62]:
# Creating X,Y Train and X,Y Validation sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
((X_train.shape, y_train.shape),(X_test.shape, y_test.shape))

(((147726, 47), (147726,)), ((63312, 47), (63312,)))

In [31]:
# Convert X_train to numpy array
arr = X_train.to_numpy()
# Reshape numpy array into 3D tensor
tarr = arr.reshape(147726, 47, 1)
# Convert ints to floats
X_train = np.asarray(tarr).astype('float32')
y_train = np.asarray(y_train).astype('float32')

# Convert X_test to numpy array
arr = X_test.to_numpy()
# Reshape numpy array into 3D tensor
varr = arr.reshape(63312, 47, 1)
# Convert ints to floats
X_test = np.asarray(varr).astype('float32')
y_test = np.asarray(y_test).astype('float32')

In [187]:
((X_train.shape, y_train.shape), (X_test.shape, y_test.shape))

(((147726, 47, 1), (147726,)), ((63312, 47, 1), (63312,)))

In [208]:
dim

131099

In [258]:
X_train

array([[[2.8000000e+01],
        [1.5000000e+03],
        [1.0000000e+01],
        ...,
        [1.0000000e+01],
        [4.4300000e+02],
        [3.2000000e+01]],

       [[4.8000000e+01],
        [3.3951188e+04],
        [8.0000000e+00],
        ...,
        [3.2000000e+02],
        [3.8000000e+01],
        [6.1000000e+01]],

       [[3.0000000e+01],
        [1.0000000e+03],
        [7.0000000e+00],
        ...,
        [7.6400000e+02],
        [1.0300000e+02],
        [3.0000000e+00]],

       ...,

       [[4.0000000e+01],
        [1.5000000e+04],
        [7.0000000e+00],
        ...,
        [8.5400000e+02],
        [2.2000000e+01],
        [6.8100000e+02]],

       [[4.4000000e+01],
        [5.0000000e+03],
        [4.0000000e+00],
        ...,
        [3.0000000e+00],
        [5.0000000e+00],
        [8.9800000e+02]],

       [[3.0000000e+01],
        [2.3473076e+03],
        [2.0000000e+00],
        ...,
        [3.8000000e+01],
        [2.0000000e+00],
        [8.7000000e+01]]

In [251]:
print('building model')
maxlen = 43
dim = len(dex)
model = Sequential()

""" Changes made
- Try dropping out unid > Holy shit it broke the model, was it keying on unid?
    Nope, I'm an idiot and it frameshifted one to left (durrrr)
- Added second LSTM layer, has not really impacted accuracy
- Added dense layer
"""

# Try embeddings (will it work if only 43/49 columns are encoded words?)

# Try normalizing inputs (just words? everything individually?)

# Try playing with layers (Add'l dense layer?  Add'l LSTM layer?)

# model.add(Embedding(input_dim=15000,
# 					embeddings_initializer="uniform", output_dim=256))

model.add(LSTM(64, input_shape=(47, 1), dropout=0.1, return_sequences=True))

model.add(LSTM(64, input_shape=(47, 1), return_sequences=True))

model.add(LSTM(64, input_shape=(47, 1), return_sequences=False))

model.add(Dense(10, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
			  metrics=['accuracy'])



building model


In [233]:
# from keras import backend as K

# # with a Sequential model
# get_3rd_layer_output = K.function([model.layers[3].input],
#                                   [model.layers[0].output])
# layer_output = get_3rd_layer_output([0])[0]

# print(layer_output)

In [252]:
print("training")
history = model.fit(tarr, y_train,
				   validation_data=(varr, y_test),
				   batch_size=128,
				   epochs=100,
				   verbose=1,
                   workers=10)

print(history)
print('---')

training
Epoch 1/100
1155/1155 [==============================] - 102s 85ms/step - loss: 0.6591 - accuracy: 0.6138 - val_loss: 0.6495 - val_accuracy: 0.6288
Epoch 2/100
1155/1155 [==============================] - 97s 84ms/step - loss: 0.6439 - accuracy: 0.6291 - val_loss: 0.6347 - val_accuracy: 0.6412
Epoch 3/100
1155/1155 [==============================] - 102s 88ms/step - loss: 0.6343 - accuracy: 0.6368 - val_loss: 0.6213 - val_accuracy: 0.6503
Epoch 4/100
1155/1155 [==============================] - 113s 98ms/step - loss: 0.6170 - accuracy: 0.6510 - val_loss: 0.6401 - val_accuracy: 0.6522
Epoch 5/100
1155/1155 [==============================] - 111s 96ms/step - loss: 0.6004 - accuracy: 0.6667 - val_loss: 0.6585 - val_accuracy: 0.6594
Epoch 6/100
 777/1155 [===================>..........] - ETA: 34s - loss: 0.5937 - accuracy: 0.6718

KeyboardInterrupt: 

In [174]:
# Save first model (get a point on the board!)
model.save('models/model2')
print('saved successfully')

INFO:tensorflow:Assets written to: models/model2/assets


INFO:tensorflow:Assets written to: models/model2/assets


saved successfully


In [165]:
import tensorflow as tf
# Convert model to tflite model (it can't be that simple)
converter = tf.lite.TFLiteConverter.from_saved_model('models')
tflite_model = converter.convert()

# Resave model
with open('model1.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
# Will take forever to run, accuracy is so-so for 3-category class
# Options are
#    1. Sample data set, 3,600,000 is too many (weak)
#    2. Set up EC2 instance, pay like 10 dollars/hour (cool, difficult)
#    3. Increase batch-size to the 10k range (dumb?)
#    4. Decrease validation size to 1/5
#                Test set size = 1,830,912 ... 1/5 = 366,182.4
# Blurb dataset is 150Char, Descrip dataset is 3,500Char
# Will this impact training time?  Maybe not?
